# Análisis de reingresos hospitalarios para pacientes con diabetes. 

**Etapas de modelamiento y hallazgos**

De acuerdo con lo planteado en la introducción de la parte exploratoria y descriptiva (Notebook en R), en el presente Notebook (Python) se compeltarán las estapas 4 y 5 del plan descrito; las cuales corresponden al **ajuste y validación de modelos**.

Aunque es cierto que potencialmente se pueden aplicar muchas técnicas a los datos, se decidió  ajustar **modelos de clasificación** con el objetivo de encontrar las variables que más contribuyan a la prediccón correcta de los reingresos en menos de 30 días. Se descartaron los **modelos de agrupación**(clustering) debido a que en este escenario ya se ha fijado una variable respuesta de interés (indicador de readmisión) lo cual hace nás apropiados los **modelos supervisados**.

In [2]:
# librerias iniciales requeridas:
from datetime import datetime
print(datetime.now())
import os
import pandas as pd
import numpy as np
import plotly.express as px

2021-01-04 21:08:29.188052


In [3]:
print(datetime.now())
from google.colab import drive
drive.mount('/content/drive')

2021-01-04 21:08:35.989689
Mounted at /content/drive


In [4]:
# Configuración de carpetas:
print(datetime.now())
os.getcwd()
os.chdir("/content/drive/Shareddrives/Books_and_Code_for_Colab_(R and Python)/ifood_technical_test/ifood_diabetes_project")
os.getcwd()

2021-01-04 21:09:17.927838


'/content/drive/Shareddrives/Books_and_Code_for_Colab_(R and Python)/ifood_technical_test/ifood_diabetes_project'

## Lectura de datos "pre-procesados"

Este archivo proviene del proceso de limpieza y exploración desarrollado en `R`

In [5]:
# Lectura de los datos limpios:
print(datetime.now())

data_cleaned=pd.read_csv('/content/drive/Shareddrives/Books_and_Code_for_Colab_(R and Python)/ifood_technical_test/ifood_diabetes_project/Data/diabetic_data_cleaned.csv',na_values="?")
data_cleaned.shape
data_cleaned.head()

2021-01-04 21:09:25.130198


,Medical_speciality,Age_,race_,Admission,Discharge,Reaction,Pri_diag,Readmit,time_in_hospital
0,Cardiology,"[30, 60)",Caucasian,Other,Home,None,Circulatory,1,8
1,Surgery,"[30, 60)",Caucasian,Other,Home,None,Musculoskeletal,0,2
2,InternalMedicine,"[60, 100)",Caucasian,Emergency,Other,Norm,Injury,0,4
3,InternalMedicine,"[60, 100)",Caucasian,Emergency,Home,High&Ch,Others,0,3
4,InternalMedicine,"[30, 60)",AfricanAmerican,Emergency,Home,None,Genitourinary,0,5


A continuación se entandariza la variable numérica `time_in_hospital` y se recodifican las demás variables caregóricas.

In [6]:
print(datetime.now())
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(data_cleaned["time_in_hospital"].values.reshape(-1, 1))
time_in_hospital_std = sc.transform(data_cleaned["time_in_hospital"].values.reshape(-1, 1))

2021-01-04 21:09:32.017266


In [7]:
## Imprimiendo el nombre de las columnas en el conjunto de datos preprocesado:
print(datetime.now())
print(list(data_cleaned.columns))


2021-01-04 21:09:35.057758
['Medical_speciality', 'Age_', 'race_', 'Admission', 'Discharge', 'Reaction', 'Pri_diag', 'Readmit', 'time_in_hospital']


A continuación se recodifican las variables **categóricas** por medio de múltiples variables **dummy**. Adicionalemte se agrega la variable nummérica previamente estandarizada, formando un nuevo conjunto de variables predictoras llamdo `X_imb` y un vecor de respuestas llamdo `y_imb`. El prefijo **_imb** se usa para indicar que las clases están desbalanceadas.

In [8]:
print(datetime.now())
X_imb = pd.concat(  [pd.DataFrame(time_in_hospital_std, columns=["time_in_hospital_std"]),     
                     pd.get_dummies(data_cleaned['Medical_speciality'], prefix='Medical_speciality', drop_first=True),\
                     pd.get_dummies(data_cleaned['Age_'], prefix='Age', drop_first=True),\
                     pd.get_dummies(data_cleaned['race_'], prefix='race', drop_first=True),\
                     pd.get_dummies(data_cleaned['Admission'], prefix='Admission', drop_first=True),\
                     pd.get_dummies(data_cleaned['Discharge'], prefix='Discharge', drop_first=True),\
                     pd.get_dummies(data_cleaned['Reaction'], prefix='Reaction', drop_first=True),\
                     pd.get_dummies(data_cleaned['Pri_diag'], prefix='Pri_diag', drop_first=True),],axis=1)

2021-01-04 21:09:39.397214


In [9]:
print(datetime.now())
y_imb=data_cleaned['Readmit']

2021-01-04 21:09:42.208342


Como se muestra a continuación, la **clase positiva** está altamente desbalanceada ($\approx\ 9\%$), por lo cual el desempeño de cualquier clasificador será engañoso. Para corregir este problema se opta por aumentar el número de casos con un muestreo aleatorio con repetición.

In [10]:
print(datetime.now())
y_imb.value_counts()#/y_imb.shape[0]


2021-01-04 21:09:44.658134


0    63705
1     6285
Name: Readmit, dtype: int64

In [14]:
print(datetime.now())
from sklearn.utils import resample

print('Número inicial de registros para la clase 1:', X_imb[y_imb == 1].shape[0])

X_upsampled, y_upsampled = resample(X_imb[y_imb == 1],
                                    y_imb[y_imb == 1],
                                    replace=True,
                                    n_samples=X_imb[y_imb == 0].shape[0],
                                    random_state=123)

print('Número actual de registros para la clase 1 :', X_upsampled.shape[0])

2021-01-04 21:11:18.710851
Número inicial de registros para la clase 1: 6285
Número actual de registros para la clase 1 : 63705


Se puede observar como se equilibró el número de registros para cada una de las dos clases

In [15]:
print(datetime.now())
X_bal = np.vstack((X_imb[y_imb == 0], X_upsampled))
y_bal = np.hstack((y_imb[y_imb == 0], y_upsampled))
print(X_bal.shape)
print(y_bal.shape)

2021-01-04 21:11:29.930638
(127410, 25)
(127410,)


## Ajuste de modelos

Se pretende construir y comparar los siguientes modelos de clasificación binaria:

* Regresión Logística 
* Máquinas de Vectores de Soporte (lineal, para poder interpretar los pesos)
* Arbol de decisión

Para poder comparar el desempeño de los modelos se utiliza la estrategia de **validación cruzada anidada**

In [16]:
## Separación de los conjunto de datos para entrenamiento y prueba:
print(datetime.now())
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X_bal, y_bal, 
                     test_size=0.20,
                     stratify=y_bal,
                     random_state=1)

2021-01-04 21:11:43.836321


### Modelo de regresión logística

Se inicia con una prueba donde aun no se usa la validación cruzada.

In [17]:
print(datetime.now())
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(LogisticRegression(random_state=12345, max_iter=1000))

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print('Train Accuracy: %.3f' % pipe_lr.score(X_train, y_train))
print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

2021-01-04 21:11:49.842704
Train Accuracy: 0.579
Test Accuracy: 0.581


Se observa un desempeño muy pobre. Para evitar la ocurrencia de un subajuste, se trabaja con una validación cruzada que internamente trata de optimizar el parámetro de regularización inversa $C$.

In [23]:
%%time
print(datetime.now())
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
gs = GridSearchCV(estimator=pipe_lr,
                  param_grid=[{'logisticregression__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}],
                  scoring='f1',
                  cv=5)

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='roc_auc', cv=10)
print('CV roc_auc: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='accuracy', cv=10)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),  np.std(scores)))

2021-01-04 06:03:02.890777
CV roc_auc: 0.606 +/- 0.007
CV accuracy: 0.579 +/- 0.005
CPU times: user 16min 27s, sys: 9min, total: 25min 27s
Wall time: 12min 55s


Los resultados muestran que el desempeño no mejoró y que tal vez es necesario considerar interacciones las variables (lo cual no se hará por el momento)

### Máquinas de Vectores de Soporte.

Como el objetivo es determinar la importancia de los factores que mejor perfilan a los pacientes readmitidos (y no solamente obtener predicciones altammente precisas) utilizamos kernels lineales para las **SVM**, debido a que con kernels de otros tipos (poly, rbf y sigmoid) los datos deben ser trasnformados a otro espacio donde los coeficientes ya no conservan su interpretación. Está bien documentado que esta técnica requiere largos tiempos de entrenamiento. 

In [ ]:
%%time
from sklearn.svm import SVC
print(datetime.now())
pipe_svc = make_pipeline( SVC(random_state=1, kernel="linear"))

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svc__C': param_range}]

# {'svc__C': param_range, 
#                'svc__gamma': param_range, 
#                'svc__kernel': ['rbf']}

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=5,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

2021-01-04 06:54:19.118294


Los tiempos de entrenamiento se tornaron **excesivemente largos** y tras 12 horas de ejecución esta primera fase de optimización para el hiperparámetro $C$ no había terminado. Se abandona la posibilidad de este modelo debido al inconveniente en los tiempos de entrenamiento; de cualquier forma según la **tabla 1** en Chopra *et al*. (2017), el modelo **SVM** apenas superó al de regresión logística en rendimiento con un $64.88\%$ de axactitud  

In [ ]:
%%time
from sklearn.svm import SVC

pipe_svc = make_pipeline( SVC(random_state=1, kernel="linear"))

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svc__C': param_range}]

gs = GridSearchCV(estimator=pipe_svc,
                  param_grid=param_grid,
                  scoring='f1',
                  cv=2)

scores = cross_val_score(gs, X_train, y_train, scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

In [ ]:
param_grid

[{'svc__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
  'svc__kernel': ['linear']},
 {'svc__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
  'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
  'svc__kernel': ['rbf']}]

### Arboles de decisión.

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                  param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7,10,15,20, None]}],
                  scoring='f1',
                  cv=5)

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='roc_auc', cv=10)
print('CV roc_auc: %.3f +/- %.3f' % (np.mean(scores),  np.std(scores)))

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='accuracy', cv=10)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),  np.std(scores)))

# print(gs.best_params_)

CV roc_auc: 0.810 +/- 0.003
CV accuracy: 0.724 +/- 0.004
Wall time: 3min 43s


El modelo de árbol de decisión mejoró el desempeño en la clasificación de manera apreciable. En este punto se tiene que a  $3$ de cada $4$ pacientes (aproximadamente) se les pude predecir de manera correcta si serán readmitidos antes de 30 días posteriores al alta médica. 

In [ ]:
gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                  param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7,10,15,20, None]}],
                  scoring='f1',
                  cv=5)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.7371216051723815
{'max_depth': None}


Ahora se ajustará el árbol con `'max_depth'= None`, usando todos los datos del conjunto de entrenamiento.

In [ ]:
# !pip install pydotplus
from io import StringIO
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image, SVG

clf = DecisionTreeClassifier(max_depth = None)

# Train Decision Tree Classifer

clf = clf.fit(X_train, y_train)

In [ ]:
feat_labels = X_imb.columns

In [ ]:
importances=clf.feature_importances_
indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,
    feat_labels[indices[f]],
    importances[indices[f]]))

 1) time_in_hospital_std           0.227079
 2) race_Caucasian                 0.094813
 3) Admission_Other                0.075396
 4) Discharge_Other                0.059969
 5) Reaction_None                  0.046318
 6) Medical_speciality_Missing     0.038662
 7) Pri_diag_Diabetes              0.036051
 8) Pri_diag_Others                0.035630
 9) Pri_diag_Digestive             0.033542
10) Medical_speciality_Other       0.033453
11) Reaction_Norm                  0.033241
12) Medical_speciality_InternalMedicine 0.033043
13) Medical_speciality_GeneralPractice 0.029450
14) Pri_diag_Respiratory           0.028861
15) race_Other                     0.027850
16) Pri_diag_Injury                0.022951
17) race_Missing                   0.021002
18) Pri_diag_Neoplasms             0.020337
19) Pri_diag_Genitourinary         0.019163
20) Reaction_High&Not              0.018878
21) Age_[60, 100)                  0.017092
22) Medical_speciality_Surgery     0.016564
23) Age_[30, 60)       

### Conclusiones:

* De la salida anteior se deduce que las variables con mayor poder discriminatorio a la hora de predecir si un paciente será reingresado son **time_in_hospital**, **race_Caucasian**, **dmission_Other** y **Discharge_Other**.


* Aunque el poder predictivo del árbol de decisión final es `aceptable`, este podría ser mejorado con un modelo de **redes neuronales recurrentes**, como lo proponen **Chopra et al.** (2017), pero con el inconveniente de ser menos interpretable en términos de la importancia de las variables.


* Chopra *et al*. (2017), realiza una comparación de varios métodos de clasificación; entre ellos Regresión logística, SVM, Árboles de decisión, Redes neuronales simples y Redes nuronales recurrentes, siendo este último método quien obtuvo el mejor comportamiento con un $AUC=0.8$ y un $ACC=81,12\%$ ........

### Referencias

* Chopra, C., Sinha, S., Jaroli, S., Shukla, A. and Maheshwari, S. (2017). **Recurrent Neural Networks with Non-Sequential Data to Predict Hospital Readmission of Diabetic Patients**. ICCBB 2017: Proceedings of the 2017 International Conference on Computational Biology and BioinformaticsOctober 2017 Pages 18–23.


* Strack, B., DeShazo, J. P., Gennings, C., Olmo, J. L., Ventura, S., Cios, K. J. and Clore, J. N. (2014). **Impact of hba1c measurement on hospital readmission rates: analysis of 70,000 clinical data base patient records**. BioMed research international. Volume 2014, Article ID 781670.


